In [1]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, DefaultDataCollator, create_optimizer
import datasets
import tensorflow as tf

In [2]:
def load_train_val_test_datasets(dataset_path='./../data/datasets/squad'):
    train = pd.read_csv(f'{dataset_path}/train.csv').dropna()
    val = pd.read_csv(f'{dataset_path}/dev.csv').dropna()
    test = pd.read_csv(f'{dataset_path}/test.csv').dropna()
    return train, val, test


def convert_dataframes_to_datasets(dataframes: list):
    return tuple(
        [datasets.Dataset.from_pandas(dataframe, preserve_index=False) for dataframe in
         dataframes])


df_train, df_val, df_test = load_train_val_test_datasets()

train_dataset, val_dataset, test_dataset = convert_dataframes_to_datasets([df_train, df_val, df_test])

c:\Users\Artur\.conda\envs\nlp_gpu\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [3]:
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 68716
 }),
 Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 14724
 }),
 Dataset({
     features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
     num_rows: 14725
 }))

In [4]:
model_checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample['question'].strip()
    context = sample['context'].strip()

    return tokenizer(
        question,
        context,
        max_length=max_tokens,
        padding=padding
    )


tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_val_dataset = val_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print('Max number of tokens in tokenized train dataset: ', len(max(tokenized_train_dataset['input_ids'], key=len)))
print('Max number of tokens in tokenized val dataset: ', len(max(tokenized_val_dataset['input_ids'], key=len)))
print('Max number of tokens in tokenized test dataset: ', len(max(tokenized_test_dataset['input_ids'], key=len)))

Map:   0%|          | 0/68716 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/14724 [00:00<?, ? examples/s]

Map:   0%|          | 0/14725 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  882
Max number of tokens in tokenized val dataset:  879
Max number of tokens in tokenized test dataset:  831


In [6]:
max_length = 384


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample['input_ids']) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(train_dataset, max_tokens=max_length)
filtered_val_dataset = filter_samples_below_number_of_tokens(val_dataset, max_tokens=max_length)
filtered_test_dataset = filter_samples_below_number_of_tokens(test_dataset, max_tokens=max_length)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x0000013DE2D84CF0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [7]:
filtered_train_dataset['question'][0]

'Along with the United Democratic Party, what party currently rules the Marshall Islands?'

In [8]:
start_char = filtered_train_dataset['answer_start'][0]
start_char

633

In [9]:
answer_text = filtered_train_dataset['answer_text'][0]
answer_text

'the AKA'

In [10]:
end_char = start_char + len(answer_text)
end_char

640

In [11]:
filtered_train_dataset['id'][0]

'56f961049b226e1400dd13eb'

In [12]:
print('Number of samples in tokenized train dataset before filtering: ', len(train_dataset))
print('Number of samples in tokenized val dataset before filtering: ', len(val_dataset))
print('Number of samples in tokenized test dataset before filtering: ', len(test_dataset))

print('\n---------------\n')

print('Number of samples in tokenized train dataset after filtering: ', len(filtered_train_dataset))
print('Number of samples in tokenized val dataset after filtering: ', len(filtered_val_dataset))
print('Number of samples in tokenized test dataset after filtering: ', len(filtered_test_dataset))

Number of samples in tokenized train dataset before filtering:  68716
Number of samples in tokenized val dataset before filtering:  14724
Number of samples in tokenized test dataset before filtering:  14725

---------------

Number of samples in tokenized train dataset after filtering:  67801
Number of samples in tokenized val dataset after filtering:  14541
Number of samples in tokenized test dataset after filtering:  14519


In [13]:
filtered_train_dataset

Dataset({
    features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start'],
    num_rows: 67801
})

In [14]:
def tokenize(examples):
    questions = [q.strip() for q in examples['question']]
    # contexts = [c.strip() for c in examples['context']]

    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=max_length,
        padding='max_length'
    )

    return inputs

b = filtered_train_dataset.map(tokenize, batched=True)
b

Map:   0%|          | 0/67801 [00:00<?, ? examples/s]

Dataset({
    features: ['index', 'id', 'context', 'question', 'answer_text', 'answer_start', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 67801
})

In [28]:
# def tokenize(examples):
#     questions = [q.strip() for q in examples['question']]
#     # contexts = [c.strip() for c in examples['context']]

#     inputs = tokenizer(
#         questions,
#         examples['context'],
#         max_length=max_length,
#         padding='max_length'
#     )

#     return inputs


def preprocess_dataset(dataset):
    questions = [q.strip() for q in dataset['question']]
    # contexts = [c.strip() for c in examples['context']]

    inputs = tokenizer(
        questions,
        dataset['context'],
        max_length=max_length,
        padding='max_length',
    )
    # sample_map = inputs.pop("overflow_to_sample_mapping")
    # for key, values in dataset.items():
    #     inputs[key] = [values[i] for i in sample_map]
    # dataset = dataset.map(tokenize, batched=True)

    answer_start_indices = dataset['answer_start']
    answer_texts = dataset['answer_text']

    start_positions = []
    end_positions = []

    for index, _ in enumerate(questions):
        start_char = answer_start_indices[index]
        end_char = start_char + len(answer_texts[index])

        start_positions.append(start_char)
        end_positions.append(end_char)

    # inputs = inputs.add_column('start_positions', start_positions)
    # inputs = inputs.add_column('end_positions', end_positions)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


# tokenized_train_dataset = preprocess_dataset(filtered_train_dataset)
# tokenized_val_dataset = preprocess_dataset(filtered_val_dataset)
# tokenized_test_dataset = preprocess_dataset(filtered_test_dataset)

In [47]:
tokenized_train_dataset = filtered_train_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_train_dataset.column_names,
)
len(filtered_train_dataset), len(train_dataset)

Map:   0%|          | 0/67801 [00:00<?, ? examples/s]

(67801, 68716)

In [48]:
tokenized_val_dataset = filtered_val_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_val_dataset.column_names,
)
len(filtered_val_dataset), len(train_dataset)

Map:   0%|          | 0/14541 [00:00<?, ? examples/s]

(14541, 68716)

In [ ]:
print(f'All tokenized train dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_train_dataset['input_ids']]))
print(f'All tokenized val dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_val_dataset['input_ids']]))
print(f'All tokenized test dataset entries have {max_length} tokens: ',
      all([len(input_ids) == max_length for input_ids in tokenized_test_dataset['input_ids']]))

All tokenized train dataset entries have 384 tokens:  True
All tokenized val dataset entries have 384 tokens:  True
All tokenized test dataset entries have 384 tokens:  True


In [49]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
data_collator = DefaultDataCollator(return_tensors='tf')

In [51]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_val_dataset = model.prepare_tf_dataset(
    tokenized_val_dataset,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

In [36]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, 384), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(8, 384), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, 384), dtype=tf.int64, name=None)}, {'start_positions': TensorSpec(shape=(8,), dtype=tf.int64, name=None), 'end_positions': TensorSpec(shape=(8,), dtype=tf.int64, name=None)})>

In [ ]:
tf_train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=['start_positions', 'end_positions'],
    batch_size=8,
    shuffle=False
)

tf_val_dataset = tokenized_val_dataset.to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=['start_positions', 'end_positions'],
    batch_size=8,
    shuffle=False
)

In [52]:
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [54]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=num_train_epochs)

Epoch 1/3
 324/8475 [>.............................] - ETA: 16:43 - loss: nan

KeyboardInterrupt: 

In [69]:
model.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108891648 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 108,893,186
Trainable params: 108,893,186
Non-trainable params: 0
_________________________________________________________________
